In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
global_step = tf.Variable(0, trainable=False, name='gloabal_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# with tf.name_scope로 묶은 블록은 텐서보드에서 한 계층 내부를 표현해줍니다.
# 변수 뒤에 name='W1'처럼 이름을 붙이면 텐서보드에서 해당 이름의 변수가 어디에서 사용되는지 쉽게 확인할 수 있습니다.
# 이름은 변수뿐만 아니라 플레이스 홀더, 각각의 연산, 활성화 함수 등 모든 텐서에 붙일 수 있습니다.
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))
    
with tf.name_scope('layer3'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3) #출력단에서는 보통 Activation Function을 사용하지 않습니다.
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    # 손실값을 추적하기 위해 수집할 값을 지정하는 코드를 작성합니다.
    tf.summary.scalar('cost', cost)

In [3]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

# tf.summary.merge_all 함수로 앞서 지정한 텐서들을 수집한 다음
# tf.summary.FileWriter함수를 이용해 그래프와 텐서들의 값을 저장할 디렉터리를 설정합니다.
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d, ' % sess.run(global_step), 'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    
    # session.run을 이용해 앞서 merged로 모아둔 텐서의 값들을 계산하여 수집한뒤,
    # writer.add_summary함수를 이용해 해당 값들을 앞서 지정한 디렉토리에 저장합니다.
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))
    
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

# Consol에서
# tensorboard --logdir=./logs
# output에 출력된 portnumber를 확인하고, 웹브라우저에서 열면 TensorBoard가 보입니다.

INFO:tensorflow:Restoring parameters from ./model/dnn.ckpt-200
Step: 201,  Cost: 0.549
Step: 202,  Cost: 0.549
Step: 203,  Cost: 0.549
Step: 204,  Cost: 0.549
Step: 205,  Cost: 0.549
Step: 206,  Cost: 0.549
Step: 207,  Cost: 0.549
Step: 208,  Cost: 0.549
Step: 209,  Cost: 0.549
Step: 210,  Cost: 0.549
Step: 211,  Cost: 0.549
Step: 212,  Cost: 0.549
Step: 213,  Cost: 0.549
Step: 214,  Cost: 0.549
Step: 215,  Cost: 0.549
Step: 216,  Cost: 0.549
Step: 217,  Cost: 0.549
Step: 218,  Cost: 0.549
Step: 219,  Cost: 0.549
Step: 220,  Cost: 0.549
Step: 221,  Cost: 0.549
Step: 222,  Cost: 0.549
Step: 223,  Cost: 0.549
Step: 224,  Cost: 0.549
Step: 225,  Cost: 0.549
Step: 226,  Cost: 0.549
Step: 227,  Cost: 0.549
Step: 228,  Cost: 0.549
Step: 229,  Cost: 0.549
Step: 230,  Cost: 0.549
Step: 231,  Cost: 0.549
Step: 232,  Cost: 0.549
Step: 233,  Cost: 0.549
Step: 234,  Cost: 0.549
Step: 235,  Cost: 0.549
Step: 236,  Cost: 0.549
Step: 237,  Cost: 0.549
Step: 238,  Cost: 0.549
Step: 239,  Cost: 0.549
S